In [1]:
!pip install morfeusz2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 35.9 MB/s eta 0:00:00


In [2]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
# from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import networkx as nx
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import morfeusz2
morf = morfeusz2.Morfeusz()

# Scraper
import string
import requests
from bs4 import BeautifulSoup

nltk.download("stopwords")
nltk.download('punkt_tab')
# nlp = spacy.load("pl_spacy_model_morfeusz")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
def scrape_text_from_url(url: string):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        h1_title = soup.find('h1', class_='title').get_text() if soup.find('h1', class_='title') else 'No title found'
        p_lead = soup.find('p', class_='lead').get_text() if soup.find('p', class_='lead') else 'No lead found'
        div_content_parts = soup.find('div', class_='contentparts')
        p_content_parts = []
        if div_content_parts:
            for p in div_content_parts.find_all('p', class_='contentpart--text'):
                text = p.get_text()
                if '\xa0' not in text and 'ZOBACZ WIDEO' not in text and not (p.find('a') and text == p.find('a').get_text()):
                    p_content_parts.append(text)
        return h1_title, p_lead, p_content_parts
    else:
        return None, 'No title found', 'No lead found'

In [4]:
import re

def find_first_match(word, analyses, verbose=False):
    if (verbose):
      print(word, analyses)

    for analysis in analyses:
        if analysis[2][0] == word:
            return analysis[2][1].split(":")[0]
    return word

def preprocess_text(text, lematize=True):
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"[(),]", "", text).strip()

    if (lematize):
      lematized = [ find_first_match(word, morf.analyse(word)) for word in text.split()]
      print(lematized)
      return " ".join(lematized)
    return text

In [5]:
preprocess_text("Coś się stało ciekawego i teraz ten tekst miałby lub takżę będzie lematyzowany Luke ja jestem twoim ojcem Litwo ojczyzno moja ")
# morf.analyse("20:40")

['coś', 'się', 'stać', 'ciekawy', 'i', 'teraz', 'ten', 'tekst', 'miałby', 'lub', 'takżę', 'być', 'lematyzowany', 'luke', 'ja', 'być', 'twoi', 'ojciec', 'litwa', 'ojczyzna', 'moja']


'coś się stać ciekawy i teraz ten tekst miałby lub takżę być lematyzowany luke ja być twoi ojciec litwa ojczyzna moja'

In [6]:
def read_article(url):
    res = scrape_text_from_url(url)
    raw_text = '. '.join((res[0], res[1], ' '.join(res[2])) )

    original_sentences = raw_text.split(". ")

    processed_sentences = [preprocess_text(sentence).split(" ") for sentence in original_sentences if sentence.strip()]

    return original_sentences, processed_sentences

    # -------------------------------------------
    # article = text.split(". ")
    # sentences = []
    # for sentence in article:
    #     sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    # return sentences
    # -------------------------------------------
    # sentences = sent_tokenize(text)  # Tokenizacja zdań
    # return [word_tokenize(sentence) for sentence in sentences]

In [7]:
def sentence_similarity(sent1, sent2):
    stop_words = stopwords.words('english')

    filtered_sent1 = [word.lower() for word in sent1 if word.lower() not in stop_words]
    filtered_sent2 = [word.lower() for word in sent2 if word.lower() not in stop_words]

    word_counts1 = Counter(filtered_sent1)
    word_counts2 = Counter(filtered_sent2)

    all_words = set(word_counts1.keys()).union(set(word_counts2.keys()))

    vector1 = [word_counts1.get(word, 0) for word in all_words]
    vector2 = [word_counts2.get(word, 0) for word in all_words]

    return 1 - cosine_distance(vector1, vector2)

# def sentence_similarity(sent1, sent2):
#     stop_words = stopwords.words('english')

#     sent1 = [w.lower() for w in sent1]
#     sent2 = [w.lower() for w in sent2]

#     all_words = list(set(sent1 + sent2))

#     vector1 = [0] * len(all_words)
#     vector2 = [0] * len(all_words)

#     # zbudowanie wektora dla pierwszego zdania
#     for w in sent1:
#         if w in stop_words:
#             continue
#         vector1[all_words.index(w)] += 1

#     # zbudowanie wektora dla drugiego zdania
#     for w in sent2:
#         if w in stop_words:
#             continue
#         vector2[all_words.index(w)] += 1

#     return 1 - cosine_distance(vector1, vector2)

In [8]:
def similarity_matrix_bow(sentences):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
              similarity_matrix[i][j] = sentence_similarity(sentences[i], sentences[j])

    return similarity_matrix


# #  Alternatywna metoda
from sklearn.feature_extraction.text import TfidfVectorizer
def similarity_matrix_tfidf(sentences, stop_words = stopwords.words('english')):
    # Połącz zdania w jeden tekst na potrzeby TF-IDF
    sentence_texts = [" ".join(sentence) for sentence in sentences]

    # Stwórz macierz TF-IDF
    tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentence_texts)

    # Wyznacz podobieństwo kosinusowe między wektorami TF-IDF
    similarity_matrix = (tfidf_matrix * tfidf_matrix.T).toarray()
    return similarity_matrix

In [9]:
kernel = 'bow'

def generate_summary(url, top_n=5):
    # Krok 1 - Odczytanie oryginalnych i przetworzonych zdań
    original_sentences, processed_sentences = read_article(url)
    if kernel == 'tfidf':
        sentence_similarity_martix = similarity_matrix_tfidf(processed_sentences)
    elif kernel == 'bow':
        sentence_similarity_martix = similarity_matrix_bow(processed_sentences)
    else:
        raise Exception("Unknown kernel")

    # Krok 3 - Uporządkowanie zdań w macierzy podobieństwa
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Sortowanie zdań według wyniku PageRank
    ranked_sentences = sorted(((scores[i], i) for i in range(len(processed_sentences))), reverse=True)

    # Krok 4 - Wybranie najlepszych zdań w oryginalnej formie
    num_sentences = min(top_n, len(ranked_sentences))
    summarize_text = [original_sentences[ranked_sentences[i][1]] for i in range(num_sentences)]

    # Zwrot oryginalnych zdań jako podsumowanie
    return ". ".join(summarize_text)


In [10]:
url = 'https://sportowefakty.wp.pl/pilka-reczna/1160547/polki-w-meczu-ze-szwecja-podjely-ryzyko-niesamowite-co-zrobila-bramkarka'
generate_summary(url, 4)

['były', 'tylko', 'moment']
['bolesny', 'lekcja', 'dla', 'polka', 'na', 'mój']
['nie', 'udać', 'się', 'sprawić', 'niespodzianka', 'Polska', 'w', 'pierwszy', 'mecz', 'faza', 'główny', 'mój', '2024', 'piłkarka', 'ręczny']
['w', 'pierwszy', 'połowa', 'biało-czerwone', 'walczyć', 'z', 'Szwecja', 'jak', 'równy', 'z', 'równe', 'ale', 'późno', 'do', 'głos', 'dojść', 'rywalka']
['z', 'on', 'kapitalny', 'bramkarka', 'na', 'czele.']
['reprezentacja', 'polski', 'piłkarka', 'ręczny', 'faza', 'główny', 'mój', '2024', 'do', 'który', 'awansować', 'po', '10', 'lato', 'przerwa', 'rozpocząć', 'od', 'starcie', 'z', 'czwarty', 'siła', 'na', 'świat']
['Szwedka', 'czyli', 'półfinalistka', 'ostatni', 'MŚ', 'i', 'io', '2024', 'w', 'Paryż', 'były', 'zdecydować', 'faworyt', 'ale', 'polka', 'nie', 'przestraszyć', 'się', 'rywalka']
['przynajmniej', 'w', 'pierwszy', 'połowa']
['możny', 'powiedzieć', 'że', 'być', 'to', 'stare', 'dobre', 'znajoma', 'ponieważ', 'pojedynek', 'w', 'Debreczyn', 'być', 'już', 'on', 'star

"Nie udało się sprawić niespodzianki Polsce w pierwszym meczu fazy głównej ME 2024 piłkarek ręcznych. Biało-Czerwone rzuciły się do odrabiania strat i w końcówce pierwszego kwadransa na tablicy wyników widniał już remis. W swoim najlepszym momencie, gdy piłkę do siatki władowała córka selekcjonera - Tyra Axner, Trzy Korony prowadziły już nawet różnicą dziesięciu bramek (32:22 w 56'). Przynajmniej w pierwszej połowie"

In [ ]:
"2 4".replace("[^a-zA-Z]", " ")

'2 4'